In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [4]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [5]:
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.classifier import StackingClassifier

In [6]:
df = pd.read_csv('heart.csv')    # loading the dataset
df.head()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [10]:
df = df.drop(['Unnamed: 0'], axis=1)

In [34]:
df.Thal.unique()

array(['fixed', 'normal', 'reversable'], dtype=object)

In [25]:
df.isnull().sum().sum()

6

In [33]:
df.isnull().values.any()

False

In [27]:
df['Thal'].isnull().sum()

2

In [32]:
df = df.dropna()

In [17]:
df = df.rename(columns={"AHD": "target"})

In [15]:
df['ChestPain'].replace(['typical', 'asymptomatic', 'nonanginal', 'nontypical'],
                        [0, 1, 2, 3], inplace=True)

In [20]:
df['target'].replace(['No', 'Yes'], [0, 1], inplace=True)

In [35]:
df['Thal'].replace(['fixed', 'normal', 'reversable'], [0, 1, 2], inplace=True)

C:\Users\vishal.desai\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [38]:
# Creating X and y for training
X = df.drop('target', axis = 1)
y = df['target']

In [39]:
# 20 % training dataset is considered for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

##### SCALING FEW COLUMNS

In [40]:
# initializing sc object
sc = StandardScaler()

# variables that needed to be transformed
var_transform = ['Thal', 'Age', 'RestBP', 'Oldpeak', 'Chol']
X_train[var_transform] = sc.fit_transform(X_train[var_transform])   # standardizing training data
X_test[var_transform] = sc.transform(X_test[var_transform])            # standardizing test data
print(X_train.head())

          Age  Sex  ChestPain    RestBP      Chol  Fbs  RestECG  MaxHR  ExAng  \
277 -1.741679    0          2  0.319284 -0.544213    0        0    152      0   
262  0.601114    0          0  0.985981 -0.161771    0        0    171      0   
30   1.605169    0          0  0.430400 -0.180893    0        0    151      0   
22   0.377991    1          3 -0.680763  0.679601    0        2    160      0   
281 -0.849186    1          2 -0.125182  0.086816    0        0    179      0   

      Oldpeak  Slope   Ca      Thal  
277 -0.915041      2  0.0 -0.603035  
262 -0.121634      1  0.0 -0.603035  
30   0.671774      1  2.0 -0.603035  
22   0.671774      2  0.0 -0.603035  
281 -0.915041      1  0.0 -0.603035  


<ipython-input-40-28d9508e7817>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[var_transform] = sc.fit_transform(X_train[var_transform])   # standardizing training data
C:\Users\vishal.desai\Anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<ipython-input-40-28d9508e7817>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [41]:
pd.DataFrame(X_train)

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal
277,-1.741679,0,2,0.319284,-0.544213,0,0,152,0,-0.915041,2,0.0,-0.603035
262,0.601114,0,0,0.985981,-0.161771,0,0,171,0,-0.121634,1,0.0,-0.603035
30,1.605169,0,0,0.430400,-0.180893,0,0,151,0,0.671774,1,2.0,-0.603035
22,0.377991,1,3,-0.680763,0.679601,0,2,160,0,0.671774,2,0.0,-0.603035
281,-0.849186,1,2,-0.125182,0.086816,0,0,179,0,-0.915041,1,0.0,-0.603035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,-0.514501,1,2,-0.180740,-1.003143,0,0,163,0,-0.915041,1,0.0,-0.603035
71,1.382045,1,1,-0.402972,0.105938,1,0,163,0,-0.738729,2,2.0,1.118884
107,0.266430,1,2,-0.236298,-0.372114,0,2,150,0,-0.562416,2,1.0,1.118884
274,0.489553,1,0,0.097051,-0.850167,0,0,162,0,-0.209790,1,2.0,-0.603035


#### INITIALIZING KNeighborClassifier and GaussianNaiveBayes

In [42]:
KNC = KNeighborsClassifier()   # initialising KNeighbors Classifier
NB = GaussianNB()              # initialising Naive Bayes

##### Training KNeighborsClassifier

In [43]:
# Training KNeighborsClassifier
model_kNeighborsClassifier = KNC.fit(X_train, y_train)   # fitting Training Set
pred_knc = model_kNeighborsClassifier.predict(X_test)   # Predicting on test dataset

In [44]:
# Evaluation of KNeighborsClassifier 
acc_knc = accuracy_score(y_test, pred_knc)  # evaluating accuracy score
print('accuracy score of KNeighbors Classifier is:', acc_knc * 100)

accuracy score of KNeighbors Classifier is: 70.0


##### Training Naive Bayes Classifier

In [45]:
model_NaiveBayes = NB.fit(X_train, y_train)
pred_nb = model_NaiveBayes.predict(X_test)

In [46]:
acc_nb = accuracy_score(y_test, pred_nb)
print('Accuracy of Naive Bayes Classifier:', acc_nb * 100)

Accuracy of Naive Bayes Classifier: 90.0


##### Implementing Stacking Classifier 

In [47]:
lr = LogisticRegression()  # defining meta-classifier

clf_stack = StackingClassifier(classifiers = [KNC, NB],
                               meta_classifier = lr,
                               use_probas = True,
                               use_features_in_secondary = True)

# use_probas=True indicates the Stacking Classifier uses the prediction probabilities
# as an input instead of using predictions classes.
# use_features_in_secondary = True indicates Stacking Classifier not only take predictions
# as an input but also uses features in the dataset to predict on new data.

In [48]:
model_stack = clf_stack.fit(X_train, y_train)   # training of stacked model
pred_stack = model_stack.predict(X_test)       # predictions on test data using stacked model

C:\Users\vishal.desai\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### Evaluating Stacking Classifier

In [49]:
acc_stack = accuracy_score(y_test, pred_stack)  # evaluating accuracy
print('accuracy score of Stacked model:', acc_stack * 100)

# Our both individual models scores an accuracy of nearly 80% and
# our Stacked model got an accuracy of nearly 84%.
# By Combining two individual models we got a significant performance improvement.

accuracy score of Stacked model: 83.33333333333334


In [50]:
model_stack = clf_stack.fit(X_train, y_train)   # training of stacked model
pred_stack = model_stack.predict(X_test)       # predictions on test data using stacked model

C:\Users\vishal.desai\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [51]:
acc_stack = accuracy_score(y_test, pred_stack)  # evaluating accuracy
print('accuracy score of Stacked model:', acc_stack * 100)

accuracy score of Stacked model: 83.33333333333334
